In [ ]:
%%capture
!pip install datasets
!pip install git+https://github.com/huggingface/transformers.git
!pip install flax
!pip install git+https://github.com/deepmind/optax.git

In [ ]:
import jax
jax.local_devices()

# For CV

In [ ]:
from transformers import FlaxResNetModel, AutoImageProcessor
from PIL import Image
import requests
from flax.training import train_state
import optax

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
import jax.numpy as jnp

In [ ]:
dataset, ds_info = tfds.load(
    name='cifar100',
    with_info=True,
    as_supervised=True,
    data_dir='/tmp/tfds'
)

In [ ]:
dataset = tfds.as_numpy(dataset)

In [ ]:
ds_train, ds_test = dataset['train'], dataset['test']

In [ ]:
print(f'train dataset: {len(ds_train)}, test_datset: {len(ds_test)}')

In [ ]:
num_labels = ds_info.features['label'].num_classes

In [ ]:
def get_train_batches(batch_size):
  ds = tfds.load(name='cifar100', split='train', as_supervised=True, data_dir='/tmp/tfds')
  ds = ds.batch(batch_size).prefetch(1)
  return tfds.as_numpy(ds)

In [ ]:
model = FlaxResNetModel.from_pretrained('microsoft/resnet-50')

In [ ]:
# ResNet parameters
jax.tree_map(lambda x: x.shape, model.params)

In [ ]:
state = train_state.TrainState.create(
    apply_fn=model.__call__,
    params=model.params,
    tx=optax.adam(1e-3),
)